# Create and Submit a Slice

On FABRIC, a slice is a collection of logically-related resources representing a single execution of an experiment (or a portion of an experiment, as multiple slices may be involved). Typically, a slice represents a connected topology of resources known as slivers. Every slice is part of one and only one project.

Creating a slice can be accomplished using the FABlib library as shown in the following Jupyter notebook.

Creating any slice starts with follows three basic steps:

- Step 1: Create the slice by calling the `fablib.new_slice()` method and passing in the slice’s name.  At this point you have a slice object that you can use to build the slice.
- Step 2: Design the slice by adding nodes, networks, and other components to the slice object. The slice that you design depends on the goals of your experiment.
- Step 4: Optionally Validate the slice for resource feasibility.
- Step 3: Submit the slice request. A slice is not instantiated until you submit the slice to the FABRIC service using the `slice.submit()` method. When you submit the slice request, FABRIC will try to find and allocate resources for your slice.   This step is quite complicated.  Larger slices will take longer to build and any slice might be impossible given currently available resources. While submitting a slice request there are several options that help you wait for your slice to build and handle exceptions in the build process. 


### FABlib API References

- [fablib.show_config](https://fabric-fablib.readthedocs.io/en/latest/fablib.html#fabrictestbed_extensions.fablib.fablib.FablibManager.show_config)
- [fablib.list_sites](https://fabric-fablib.readthedocs.io/en/latest/fablib.html#fabrictestbed_extensions.fablib.fablib.FablibManager.list_sites)
- [fablib.new_slice](https://fabric-fablib.readthedocs.io/en/latest/fablib.html#fabrictestbed_extensions.fablib.fablib.FablibManager.new_slice)
- [slice.add_node](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.add_node)
- [slice.validate](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.validate)
- [slice.submit](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.submit)
- [slice.get_nodes](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.get_nodes)
- [slice.list_nodes](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.list_nodesß)
- [slice.show](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.show)
- [node.execute](https://fabric-fablib.readthedocs.io/en/latest/node.html#fabrictestbed_extensions.fablib.node.Node.execute)
- [slice.delete](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.delete)


## Setup the Experiment

### Import the FABlib API

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                         
fablib.show_config();

## Option 1: Create a Slice - Basic

The easiest way to submit a slice is to call the submit method on your slice without any parameters.  By default, this will block and wait until your slice is built, management access is enabled via ssh, and all dataplane interfaces have been tested for connectivity.  In addition,  the progress of your slice’s build process will be printed.  



In [ ]:
# Create a slice
slice = fablib.new_slice(name="MySlice1")

# Design the slice
node = slice.add_node(name="Node1")

#Submit the Request
slice.submit();

## Option 2: Create a Slice - Wait Silently

You can disable the progress reporting by setting `progress=False`.


In [ ]:
# Create a slice
slice = fablib.new_slice(name="MySlice2")

# Design the slice
node = slice.add_node(name="Node1")

#Submit the Request
slice_id = slice.submit(progress=False)

print(f"slice_id: {slice_id}")

## Option 3: Create a Slice - Non-blocking Submit

You can create a slice without blockng by setting `wait=False`.  Note that if you do not wait for the slice, you will likely want to manually test your slice's state before trying to use it.  

In [ ]:

# Create a slice
slice = fablib.new_slice(name="MySlice3")

# Design the slice
node = slice.add_node(name="Node1")

# Submit the request
slice.submit(wait=False)

# Save the slice ID
slice_id = slice.get_slice_id()

print(f"slice_id: {slice_id}")

Test your slice's state

In [ ]:
#Get Slice
slice = fablib.get_slice(slice_id=slice_id)

#Wait for ssh to be active
slice_id = slice.wait_ssh(progress=True)

print(f"slice_id: {slice_id}")

#Run post boot config and dataplane network tests
slice.post_boot_config();

## Option 4: Create a Slice - Requesting resources in future

You can reserve resources for future use by specifying a start time, even if they are not currently available. For example, if `NIC_ConnectX_5` or `NIC_ConnectX_6` are not presently available, you can now check their future availability and schedule them for a later time.

### Query Available Sites in Future

List available sites with `NIC_ConnectX_5` or `NIC_ConnectX_6` one day from now. Chooose a site at random from one of the available sites.

In [ ]:
from datetime import datetime
from datetime import timezone
from datetime import timedelta

start = (datetime.now(timezone.utc) + timedelta(days=1))

In [ ]:
fields=['name','nic_connectx_5_available','nic_connectx_6_available']

In [ ]:
smart_nic_sites = fablib.list_sites(fields=fields, start=start)

smart_nic_sites = smart_nic_sites.data['Name'].values.tolist()
print(f'All sites with NIC_ConnectX_5/NIC_ConnectX_6 available: {smart_nic_sites}')

In [ ]:
import random

if len(smart_nic_sites)==0:
    print('Warning - no sites with available NIC_ConnectX_5/NIC_ConnectX_6 found')
else:    
    print(f'Selecting a site at random among {smart_nic_sites}')
    site = random.choice(smart_nic_sites)

    print(f"Site chosen: {site}")

### Request a slice in Future

A slice can be created for future use with a lease starting in future by passing `lease_start_time` in `submit`. When successful, the slice will enter the `AllocatedOK` state. When the lease becomes current, the resources will be provisioned and enter the `Active` state, causing the slice to transition to the `StableOK` state.

In [ ]:
# Create a slice
slice = fablib.new_slice(name="MySlice4")

# Design the slice
node = slice.add_node(name="Node1", site=site)
node.add_component(model="NIC_ConnectX_6", name='nic1')
node.add_component(model="NIC_ConnectX_6", name='nic2')

# Submit the request
slice.submit(lease_start_time=start)

# Save the slice ID
slice_id = slice.get_slice_id()

print(f"slice_id: {slice_id}")

## Additional Options: Validate Slice

Validate your slice before submitting to check for feasibility of resources. The validation can be done in following two ways:
- Option 1: Validate the slice as resources are added to it
- Option 2: Build the topology and validate the slice

### Option 1: Validate the slice as resources are added to it

The following code provides an example of validating Nodes in two different ways:

- Add `Node1` and validate it by setting `validate=True` in `add_node`, and use `raise_exception=False` to remove the node if it is invalid without throwing an exception.
- Add `Node2` and validate it by setting `validate=True` in `add_node`, and use `raise_exception=True` to throw an exception if the node is invalid.

Doing a `list_nodes` can confirm that `Node1` is removed from the slice, but `Node2` still exists in the slice as requested.

In [ ]:
# Create a slice
slice = fablib.new_slice(name="MySlice")

In [ ]:
# Add a node and validate it by specifying , do not raise exception in case of errors and remove it from the topology 
# Node1 is requested on a host not belonging to the site and hence is invalid

node1 = slice.add_node(name="Node1", site="MAX", host="gpn-w1.fabric-testbed.net", validate=True, raise_exception=False)

In [ ]:
# Add a node and validate it, and raise exception in case of errors

# Node2 is requesting T4 and RTX600 which is an infeasible requests as none of the hosts have both type of the GPUs available.
# Since we requested Exception to be raised in case of errors, the following code raises Exception

node2 = slice.add_node(name="Node2", site="MAX", validate=True, raise_exception=True)
node2.add_component(model='NIC_Basic', name='nic2')
node2.add_component(model='GPU_RTX6000', name='gpu1')
node2.add_component(model='GPU_TeslaT4', name='gpu3')

In [ ]:
# Node1 was removed from the topology as requested and Node2 still exists in the topology as requested
slice.list_nodes();

### Option 2: Build the topology and validate the slice

We build a topology by requesting `Node1` on a host that does not belong to the site and `Node2` with two different models of GPUs. Both of these requests are invalid, and their validation by `slice.validate()` throws an exception, providing appropriate error information.

In [ ]:
# Create a slice
slice = fablib.new_slice(name="MySlice")

# Add a node
node1 = slice.add_node(name="Node1", site="MAX", host="gpn-w1.fabric-testbed.net")
# Here node1 is invalid request would be errored and removed from the slice

node2 = slice.add_node(name="Node2", site="MAX")
node2.add_component(model='NIC_Basic', name='nic2')
node2.add_component(model='GPU_RTX6000', name='gpu1')
node2.add_component(model='GPU_TeslaT4', name='gpu3')
# Here node2 is invalid request as you can not have T4 and RTX600 both on a single worker

# This call will report both the errors identified above
slice.validate()

## Additional Options: Timeouts and Retry Interval

For any blocking call you can set and overall `timeout` and a retry `interval`.  Both settings are in seconds.

In [ ]:
# Create a slice
slice = fablib.new_slice(name="MySlice5")

# Design the slice
node = slice.add_node(name="Node1")

#Submit the Request
slice_id = slice.submit(wait_timeout=600, wait_interval=60)

print(f"slice_id: {slice_id}")

In [ ]:
#Delete Slices

try:
    fablib.delete_slice("MySlice1")
except:
    pass

try:
    fablib.delete_slice("MySlice2")
except:
    pass

try:
    fablib.delete_slice("MySlice3")
except:
    pass

try:
    fablib.delete_slice("MySlice4")
except:
    pass

try:
    fablib.delete_slice("MySlice5")
except:
    pass
